#  10. Solidity 함수

* Last Updated 20190612WEB1200

## 10.1 학습내용

## 10.1.1 목표
 
* solidity 함수

## 10.1.2 목차

* 10.1 생성자
* 10.2 함수 타잎
    * 10.2.1 view, pure 함수
    * 10.2.2 Payable 함수
    * 10.2.3 가시성
    * 실습 FunctionTest
* 10.3 modifier
    * 실습 UnderscoreTest
    * 실습 BankV3
* 10.4 Events and logs
    * 실습 testEvent
    * 실습 e_testDeposit
    * 실습 e_testDeposit
    * 문제 Multiply7Event.sol
    * 실습 e_testEvent 복수의 이벤트 사용하기
* 10.5 fallback
    * 실습 FallbackTest
    * 실습 Payable Fallback - Multiply7.sol

## 10. 함수

함수는 **제공되거나 built-in** 또는 **사용자가 정의 user-defined** 로 구분할 수 있다.

아래의 문법에 맞추어, 앞에 ```function```이라고 적어서 함수 선언을 한다.
함수의 입력과 출력을 선언한다.
가시성과 수행하는 기능의 성격에 따라 ```pure```, ```view```, ```payable```의 수식어를 붙여준다.

```python
function (param types) {internal|external} [pure|view|payable] [returns (return types)] varName;
```

## 10.1 생성자

생성자 ```constructor()```는 컨트랙의 객체를 생성한다.
컨트랙의 소유자를 설정하거나, 처음에 설정이 필요한 변수를 초기화할 경우 사용한다.
생성자는 객체를 생성할 때 (마이닝해서 배포되는 시점) 한 번만 실행이 된다.
* 생성자는 구현하지 않아도 되며, 그럴 경우 기본 생성자가 주어진다. 기본 생성자에는 물론 arg가 없다.
* 생성자는 1개만 허용된다. overloading은 할 수 없다는 의미이다.
* 보통 외부에서 사용할 수 있도록 **public**으로 선언한다.
또는 **internal**로 선언하여 다른 컨트랙이 생성할 수 있게 한다.

```java
contructor() public {}
```


## 10.2 함수 타잎

### 10.2.1 view, pure 함수

보통 함수는 입출력에 따라 구분할 수 있다. 입력이 있는지, 반환 값이 무엇인지에 따라 구분할 수 있다.
블록체인 프로그램에서는 함수가 블록체인을 변경하는지에 따라 구분하는 것이 중요하다.

구분 | 설명
----------|----------
블록체인을 변경하는 view, pure가 아닌 함수 | return을 지원하지 않음. event를 사용해서 값을 돌려받음
view, pure 함수 | return 가능.

상태를 변경하지 않는 함수는 ```view```, ```pure```로 표기해야 하고, 함수를 local에서 실행할 수 있게 한다.

* ```view```는 이전 ```constant```를 대체한 명령어로서, 읽기 전용 read onlhy 함수이다.

* ```pure```: ```view```보다 더 제약이 많은 함수에 붙이는 수식어이다. 함수의 지역변수만을 사용하며, 함수 밖의 그 어떤 변수를 수정하지 못한다. 예를 들면: 
    * 상태변수 state variables를 읽거나
    * 잔고 ```this.balance``` 또는 ```<address>.balance```를 읽는 경우
    * 전역변수 ```block```, ```tx```, ```msg``` (```msg.sig```, ```msg.data``` 제외)를 읽는 경우
    * ```pure```로 표기되지 않은 함수를 호출하는 경우
    * ```opcodes```로 작성된 인라인 코드

* 블록체인을 수정하는 경우, transaction이 발생하므로 Transaction hash가 반환된다.
    - 상태변수에 저장
    - 이벤트 발생 (이벤트가 발생하면 로그에 기록이 남겨진다.)
    - 다른 컨트랙 생성
    - ```selfdestruct``` 컨트랙을 삭제하는 경우
    - ```call``` 함수로 송금하는 경우
    - ```view``` 또는 ```pure``` 아닌 함수를 호출하는 경우
    - low-level calls을 호출하는 경우
    - ```opcodes```로 작성된 인라인 코드


### 10.2.2 Payable 함수

* ```payable```: 함수를 호출할 경우 Ether를 지급해야 실행이 되는 함수.
web3 호출하는 측에서 { value: 0 }에 ether를 입력하면 ```msg.value```로 전달된다.
컨트랙에서 호출하는 경우는 value()

### 10.2.3 가시성

* ```public``` 외부에서 누구나 사용할 수 있다.
* ```internal``` 기본default 가시성으로, 내부 또는 상속관계에서만 사용할 수 있다.
* ```private``` 자신만 사용할 수 있다.
* ```external```은 ```internal```의 반대로서 외부에서만 사용할 수 있다.

컨트랙 내에 함수를 배치할 때는 다음과 같은 순서로 적어준다 (참조 Solidity Style Guide)
- 생성자
- fallback 함수
- external 함수
- public 함수
- internal 함수
- private 함수

줄 | 설명
-----|-----
5 ~ 7 ```increatementX()``` | state var를 수정하므로 view, pure가 아님.
9 ~ 11 ```doubleX()``` | 내부함수 ```internal X2()```를 호출함. state var 수정하므로 view, pure 아님.
13 ~ 15 ```getX_()``` | view, pure가 아니므로```returns(int)```해도 반환 값을 받지 못함. 해시값을 돌려줌.
19 ~ 21 ```getBalance()``` | 잔고 조회는 view도 가능함.
23 ~ 24 ```deposit()``` | ```payable```로 선언해야 입금이 가능. 함수가 비워있어도 msg.value를 입금.
26 ~ 28 ```X2()``` | ```internal```로 선언되어서 내부에서만 사용가능.

In [1]:
%%writefile src/FuctionTest.sol
pragma solidity ^0.5.0;
contract FunctionTest {
    int x=1;
    // updating state var x
    function incrementX() public {
        x+=1;
    }
    // updating state var x
    function doubleX() public {
        X2();
    }
    // actually NONE returned
    function getX_() public returns(int) {
        return x;        
    }
    function getX() view public returns(int) {
        return x;
    }
    function getBalance() view public returns(uint) {
        return(address(this).balance);
    }
    // 'payable' means that msg.value can be deposited
    function deposit() public payable {
    }
    // can not be used in public
    function X2() internal {
        x*=2;
    }
}

Writing src/FuctionTest.sol


## 10.3 modifier

함수의 **제약조건**으로 사용된다. 즉 어떤 함수가 실행되려면 만족해야 하는 조건을 modifier로 정할 수 있다. Owner인 경우, 일정한 기간에 포함되는 경우, 짝수에 해당하는 경우 등 조건을 걸어 그런 경우에만 함수를 실행할 수 있게 한다. 그 효과로 **if문을 제거하는 효과**가 있게 된다.

또한 modifier는 보통 함수가 그런 것처럼 **상속**이 가능하다. 또한 override할 수도 있다.


modifier를 사용할 때 underscore ```_``` 명령어를 통해 어디에 호출 소스코드가 포함되는지 분명히 할 수 있다.
아래 코드에서 포함되는 시점을 전 또는 후로 정할 수 있다.

In [2]:
%%writefile src/UnderscoreTest.sol
pragma solidity ^0.5.0;
contract UnderscoreTest {
    string season="";
    modifier setSummerAfter() {
        season="summer";
        _;
    }
    function setWinter() public setSummerAfter {
        season="winter";
    }
    modifier setSummerBefore() {
        _;
        season="summer";
    }
    function setSpring() public setSummerBefore {
        season="spring";
    }
    function getSeason() view public returns(string memory){
        return season;
    }
}

Writing src/UnderscoreTest.sol


## 실습: 은행

앞의 은행 코드에 fallabck, modifier를 넣어서 수정한다.

### 단계 1: 컨트랙 개발

입금을 연달아 할 경우, 반드시 일정시간이 지나야 (10초) 입금을 가능하게 하고
일정 잔고 이상일 경우에만 출금이 되도록 해보자.

In [17]:
%%writefile src/BankV3.sol
pragma solidity ^0.5.0;

contract BankV3 {
    address owner;
    uint256 timeToDeposit;
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner=msg.sender;
    }
    function() external {
        emit PrintLog("Fallback called");
    }
    function sendTo(address payable _receiver) public payable onlyOwner {
        //require(msg.sender==owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function queryBalance() public view returns(uint) {
        return address(this).balance;
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit=now+10 seconds;
        require(msg.value==amount);
    }
    function withdraw() public onlyOwner minBalance {
        //require(msg.sender==owner);
        msg.sender.transfer(address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender==owner);
        _;
    }
    modifier onlyAfter {
        require(now>=timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Overwriting src/BankV3.sol


### 단계 2: 컴파일

In [18]:
!solc --gas --abi --bin src/BankV3.sol


======= src/BankV3.sol:BankV3 =======
Gas estimation:
construction:
   20523 + 211800 = 232323
external:
   fallback:	1795
   deposit(uint256):	20509
   queryBalance():	601
   sendTo(address):	infinite
   withdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610423806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100d35780633ccfd60b146100fe578063b6b55f2514610115578063e6d2524514610143575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e8610187565b6040518082815260200191505060405180910390f35b34801561010a57600080fd5b506101136101a6565b005b6101416004803603

### 단계 3: 컨트랙 배포

In [19]:
%%writefile src/BankV3Deploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610423806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100d35780633ccfd60b146100fe578063b6b55f2514610115578063e6d2524514610143575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e8610187565b6040518082815260200191505060405180910390f35b34801561010a57600080fd5b506101136101a6565b005b6101416004803603602081101561012b57600080fd5b8101908080359060200190929190505050610287565b005b6101856004803603602081101561015957600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506102b3565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561020157600080fd5b60653073ffffffffffffffffffffffffffffffffffffffff163111151561022757600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610284573d6000803e3d6000fd5b50565b600154421015151561029857600080fd5b600a420160018190555080341415156102b057600080fd5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561030e57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f19350505050158015610354573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a15056fea165627a7a723058201a36146d11a6f9cedca78eadc10bd6bb60f95c3202eb8ccd0253fb8389e02cbc0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Overwriting src/BankV3Deploy.js


In [20]:
!node src/BankV3Deploy.js

contractAddress:  undefined
transactionHash:  0xe1ed3107a7242e90a9a09ac3ae04ce3dc5413bc73e1e7c921b87a58546b5784a
contractAddress:  0x486d4a274e03639c2bd52369f38e15736d760ee0
transactionHash:  0xe1ed3107a7242e90a9a09ac3ae04ce3dc5413bc73e1e7c921b87a58546b5784a


### 단계 4: 사용

1. deposit(100);
2. withdraw() ---> 실패 (프로그램에 101 wei 이상 제약조건)
3. deposit(111);
value를 반드시 함수인자와 동일하게 입력해야 한다.
REMIX에서 할 경우에도 마찬가지이다. Run 탭 상단의 value, Deployed Contracts 함수의 인자 두 필드에 동일한 금액을 넣어준다.
4. 빠르게 (프로그램에 설정해 놓은 10초 이내) deposit(111) ---> 실패
5. queryBalance() --=> 211
6. deposit(111); ---> 10초가 지났으면 실행. 이때 마이닝을 해서 동기화를 시켜줄 필요가 있다.
7. queryBalance() ---> 322

```python
> bank.deposit(100,{from:web3.eth.accounts[0], value:100});          입금 100
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
> bank.queryBalance().toNumber();                                    입금금액 조회 100. 앞 입금거래를 마이닝하고 금액 증가.
100
> bank.deposit(100,{from:web3.eth.accounts[0], value:111});          입금 'value'와 '인자'가 서로 다르면 입금 실패
'0x1614f5f9efc887cfa17c53c2f93f62f95bc22d6673be8db9cc8de086a70a1415'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
undefined
> bank.queryBalance().toNumber();                                    실패하였으므로 잔고는 계속 100
100
> bank.deposit(111,{from:web3.eth.accounts[0], value:111});          입금 111
'0xcc795834d708da090362217c96972e905a42280f61be3a4cd0b25f487313eaea'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
undefined
> bank.queryBalance().toNumber();                                    입금 100+111 = 211
211
> bank.deposit(111,{from:web3.eth.accounts[0], value:111});          (10초 지나서) 입금 111, 마이닝하면 금액 증가함
'0x07579161cc6ac7f8257a0c08cb63124d0355454f71300a9820b7f01abd7fd0a7'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> bank.queryBalance().toNumber();                                    입금 100+111+111 = 322
322
> bank.deposit(111,{from:web3.eth.accounts[0], value:111});          입금 111, 마이닝하면 금액 증가함
'0x9178978322503cd6b63a4aecca9f2878ff039c26f64c2fcd7a74b8950d8c8426'
> bank.deposit(111,{from:web3.eth.accounts[0], value:111});          입금 111, 10초 이내이므로 마이닝해도 금액 증가하지 않음
'0xe0e229977b416bead8bb99a8ec5d3b5519095519d612e302b6f1af7225e7b25b'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> bank.queryBalance().toNumber();                                    여러번 입금해도 10초 이내 거래는 실패하고 잔고가 늘지 않음
433
>
```

In [33]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0x486d4a274e03639c2bd52369f38e15736d760ee0");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.deposit(100, {from:web3.eth.accounts[0], value:100}));
console.log(bank.withdraw({from:web3.eth.accounts[0]}));
console.log(bank.queryBalance().toNumber());

Overwriting src/BankV3Use.js


In [34]:
!node src/BankV3Use.js

0x52bd51f84b47f4c0d7e64b340f8046042c0847975f21ff4daa4392943fd95a33
0x7549b16332f2f04e0239055a6d72e0bf228de55e951c9106490aa45c4137d35c
0


In [35]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0x486d4a274e03639c2bd52369f38e15736d760ee0");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.queryBalance().toNumber());
console.log(bank.deposit(111, {from:web3.eth.accounts[0], value:111}));
console.log(bank.withdraw({from:web3.eth.accounts[0]}));
console.log(bank.queryBalance().toNumber());

Overwriting src/BankV3Use.js


In [36]:
!node src/BankV3Use.js

0
0x45e0beba2ac003f71dc89cd59a52f7b719263f53fde603f01e604ae3e60b95a5
0x67227397881ccfa6e5768dad2a6934f186153832da02b75a1c638551049d2ef3
0


In [31]:
!node src/BankV3Use.js

0
0x59bc83487f0649e6f8911a4471a5d6ca7fd7cc390228df3f14dacce86b92ec6a
0xad6aa7b6bb063842703d01a8d463809aae68b0487fa497951cc0e33c0ebed968
0


In [29]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0x486d4a274e03639c2bd52369f38e15736d760ee0");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.queryBalance().toNumber());
console.log(bank.deposit(111, {from:web3.eth.accounts[0], value:111}));
console.log(bank.withdraw({from:web3.eth.accounts[0]}));
console.log(bank.queryBalance().toNumber());

Overwriting src/BankV3Use.js


In [21]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0x486d4a274e03639c2bd52369f38e15736d760ee0");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.queryBalance().toNumber());
console.log(bank.deposit(111, {from:web3.eth.accounts[0], value:111}));
console.log(bank.withdraw({from:web3.eth.accounts[0]}));
console.log(bank.deposit(100, {from:web3.eth.accounts[0], value:100}));
console.log(bank.deposit(100, {from:web3.eth.accounts[0], value:100}));
console.log(bank.queryBalance().toNumber());

Overwriting src/BankV3Use.js


In [22]:
!node src/BankV3Use.js

0xa39b02e2d17c858a48d2476ff0e8ddc40ae83caf20cafde7a5d9a59ffbb47fe1
0x30dfc0af0e7980800dcb4e513b05a9b3e453c8742ac06d5d929f1f5437159816
0xd6372b90ade93ed7d21b5c8374352a8498fb7f251e5a08b66fe3d500a774bf4e
0x8997df1e38ffdec571a26040e69baf6880f1eab215671f4c627f15ed5a4613dc
0


In [23]:
!node src/BankV3Use.js

0x8a050d3bc20965e150e32d4d1514b8a507a701c082e821db774c3a54daddbbc5
0x2d1a6ddb57dcc7f4ef5ed7958285ed925a602428814124a5cf247809eccbed4a
0xd3f9486038cc655f88bee5fe956f594edc67828f4a28a90ce4896d29402f2143
0x2646757448ff0d9f61732bca95c07927d3ffb4c0f51a8fff2a756e47a78173ab
100


In [24]:
!node src/BankV3Use.js

0xc61e70340e50db31a1154b5270f06ae3c6d1ee1e7244ae8b8cd5dee67c9d6c56
0x704561d0842b158cd5bf680f0c5b7ca5064b4a8961ee31015f58559d599df760
0x71849d0a0bcf8e86048d9274b2aa0ad15088e28b75ba87da57894651efb09010
0x0982afc4c441632f96c60c6ffb69661c01ac61585d0329bcf252ed98d19368ba
0


## 10.4 Events and logs
<a id='S.3.5'></a>

이벤트는 사용자가 마우스 또는 키보드에서 어떤 입력을 하거나, 센서로부터 발생할 수 있다.
블록체인에서는 그 특성 상 마우스나 키보드의 이벤트가 발생할 수 없고, ```emit```명령어로 **특정 시점에 이벤트를 발생**하도록 프로그램한다.
이벤트는 상속할 수 있어서, 부모가 정의한 이벤트를 자식이 물려받아 발생시킬 수 있다.

* 이벤트는 우선 (1) 등록이 되어야 하고, 그 후 (2) 발생하면 호출된다.
    * EVM의 로그 기능을 사용한다. 로그는 blockchain에 저장된다.
    * 함수에 listener를 설정할 수 있다.
    * JavaScript callbacks을 사용하여 이벤트가 호출되는지 리스닝한다.

단계 | 설명
-------|-------
개발-컴파일 | event를 넣은 소스를 컴파일해서 abi, bin을 구한다.
배포 | 마이닝하고 주소를 배정받기 기다린다.
사용 | 이벤트 호출, ```_instance.MyLog()```, 인자가 있어도 생략할 수 있다.
사용 | 이벤트를 가지고 있는 함수 호출, ```_instance.MyFunction()``` or ```_instance.MyEvent.watch()```

* 오류
    * "invalid adress error" default address가 설정되지 않으면, 누가 이벤트를 호출하는지 알 수 없다.
```python
> web3.eth.defaultAccount=web3.eth.accounts[1];
```

#### 단계

단계 1. 이벤트 설정
CapWords 스타일로 단어 첫 글자를 대문자로 적어준다.
인자를 넣어줄 수 있다.
인자 3개까지 로그에서 인덱스로 사용할 수 있고,
인자의 해시 값이 로그의 ```topic```으로 출력된다.
```python
event PrintLog()
```

단계 2. 함수를 호출할 때 이벤트가 발생하도록 연결한다. 이런 연결을 바인딩 **binding**이라고 한다.
```python
function fireEvent() {
    emit PrintLog()
    ...
}
```

단계 3. 이벤트의 발생을 callback 함수로 리스닝한다.
```python
1) 이벤트를 생성하고 리스닝한다.
> var myEvent = myInstance.PrintLog({from: web3.eth.accounts[0]}, {
    fromBlock: 0, 
    toBlock: 'latest'
  });
> myEvent.watch(function (error, result) {
    if (!error) {
        console.log("myEvent: event triggered ===> ",result);
        process.exit(1);
    }
});
또는 한 명령어로 합쳐서 다음과 같이 리스닝할 수 있다.
var myEvent = myInstance.PrintLog(function(error, result) {
    if (!error)
        console.log(result);
});
```

단계 4. 함수호출로 이벤트 발생

```python
> myInstance.fireEvent();
```

단계 5. 이벤트 watch 중지
```python
> myEvent.stopWatching();
```

## 예제: 간단한 이벤트

이벤트를 만들고 함수가 호출되는 시점에 발생하도록 해보자.
이벤트는 발생하는 시점에 "Hello World!" 문자열이 출력되도록 한다.
이벤트가 발생하는지는 클라이언트에서 리스닝해서 알아 낸다.
이벤트가 발생하면 로그에 기록이 되고, 리스닝한다는 것은 이런 로그의 기록에 이벤트가 발생했는지를 확인하는 것이다.

### 단계 1: 컨트랙 개발

In [6]:
%%writefile src/testEvent.sol
pragma solidity ^0.5.0;
contract testEvent {
    event MyLog(string);
    function MyFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/testEvent.sol


### 단계 2: 컴파일

In [7]:
!solc --abi src/testEvent.sol


======= src/testEvent.sol:testEvent =======
Contract JSON ABI 
[{"constant":false,"inputs":[],"name":"MyFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"MyLog","type":"event"}]


In [10]:
!solc --bin src/testEvent.sol


======= src/testEvent.sol:testEvent =======
Binary: 
6080604052348015600f57600080fd5b5060e88061001e6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063ad6e81d214604a575b600080fd5b60506052565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea165627a7a72305820da9dac7ab71e948c4150d99274d7b603a5877bd7f86eb48ce48f0f58aad0ba280029


In [11]:
!solc --gas src/testEvent.sol


======= src/testEvent.sol:testEvent =======
Gas estimation:
construction:
   99 + 46400 = 46499
external:
   MyFunction():	1753


### 단계 3: 컨트랙 배포

In [29]:
%%writefile src/testEvent1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
//copy abi from '!solc --abi src/testEvent.sol'
var _abiArray=[{"constant":false,"inputs":[],"name":"MyFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"MyLog","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);
//copy bin from '!solc --bin src/testEvent.sol'
var _bin="0x"+"6080604052348015600f57600080fd5b5060e88061001e6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063ad6e81d214604a575b600080fd5b60506052565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea165627a7a72305820da9dac7ab71e948c4150d99274d7b603a5877bd7f86eb48ce48f0f58aad0ba280029";
var _contract = web3.eth.contract(_abiArray);
var _instance=_contract.new({data:_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
  if (!err && contract.address)
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});
console.log("after async - transactionHash: ",_instance.transactionHash)
console.log("after async - contractAddress: ",_instance.address);

Overwriting src/testEvent1.js


Asynch로 호출하기 때문에 ```after async```가 먼저 출력이 된다.

In [30]:
!node src/testEvent1.js

after async - transactionHash:  null
after async - contractAddress:  undefined
transactionHash:  0xd7fdf422500077d976984b65a8f3b7eac778c25822a2724c746ca0f7140e0451
contractAddress:  0x40dc58fc998d6ed4a384e01d89ba297d5bea2863
transactionHash:  0xd7fdf422500077d976984b65a8f3b7eac778c25822a2724c746ca0f7140e0451


* transactionHash를 가지고 contractAddress를 찾는다.

* transactionHash:  0xd7fdf422500077d976984b65a8f3b7eac778c25822a2724c746ca0f7140e0451
* contractAddress:  0x40dc58fc998d6ed4a384e01d89ba297d5bea2863


In [31]:
!geth --exec 'eth.getTransactionReceipt("0xd7fdf422500077d976984b65a8f3b7eac778c25822a2724c746ca0f7140e0451");' attach http://117.16.44.45:8445

{
  blockHash: "0xf4f15ae4b0de41140d50dde6504fbade1e341e25ee018799384c334865e9fc3d",
  blockNumber: 100,
  contractAddress: "0x40dc58fc998d6ed4a384e01d89ba297d5bea2863",
  cumulativeGasUsed: 113667,
  from: "0x21c704354d07f804bab01894e8b4eb4e0eba7451",
  gasUsed: 113667,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0xcf1d6bde9f770e85e3e42497056adfcb4f1e780f04f17bee2991007aa2c9b281",
  to: null,
  transactionHash: "0xd7fdf422500077d976984b65a8f3b7eac778c25822a2724c746ca0f7140e0451",
  tran

### 단계 4: 사용


In [1]:
%%writefile src/testEvent2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[],"name":"MyFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"MyLog","type":"event"}];
var _contract=web3.eth.contract(_abiArray);
var _address="0x40dc58fc998d6ed4a384e01d89ba297d5bea2863";
var _instance=web3.eth.contract(_abiArray).at(_address);
var filter = _instance.MyLog(function (error, result) {
  if (!error)
    console.log(result);
});
_instance.MyFunction.sendTransaction({from:web3.eth.accounts[0]});

Overwriting src/testEvent2.js


In [ ]:
!node src/testEvent2.js

{ address: '0x40dc58fc998d6ed4a384e01d89ba297d5bea2863',
  blockNumber: 103,
  transactionHash: '0xc98b2b49f475eee6a590821e4c9b1c223fb8f8863b3ff075efb3d889a9f80f3b',
  transactionIndex: 0,
  blockHash: '0xaf60f519f7496773f8c841e7ef4772e2080126300f1a205cb95ce43b98c24a6a',
  logIndex: 0,
  removed: false,
  event: 'MyLog',
  args: { '': 'Hello World!' } }


## 실습: Deposit

### 단계 1: 컨트랙 개발

In [5]:
%%writefile src/e_testDeposit.sol
pragma solidity ^0.5.0;

contract e_testDeposit {
    event Deposit(
        address indexed _from,
        bytes32 indexed _id,
        uint _value
    );
    function deposit(bytes32 _id) public payable {
        emit Deposit(msg.sender, _id, msg.value);
    }
}

Overwriting src/e_testDeposit.sol


### 단계 2: 컴파일

In [6]:
!solc --abi src/e_testDeposit.sol


======= src/e_testDeposit.sol:e_testDeposit =======
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"_id","type":"bytes32"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":true,"name":"_id","type":"bytes32"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"Deposit","type":"event"}]


In [7]:
!solc --bin src/e_testDeposit.sol


======= src/e_testDeposit.sol:e_testDeposit =======
Binary: 
6080604052348015600f57600080fd5b5060e78061001e6000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063b214faa514603e575b600080fd5b606760048036036020811015605257600080fd5b81019080803590602001909291905050506069565b005b803373ffffffffffffffffffffffffffffffffffffffff167f19dacbf83c5de6658e14cbf7bcae5c15eca2eedecf1c66fbca928e4d351bea0f346040518082815260200191505060405180910390a35056fea165627a7a72305820b8fc5d5b94e85b71dc3a6ce1672f4d4a7a24d46f3447825a14203c6f08f489a80029


In [8]:
!solc --gas src/e_testDeposit.sol


======= src/e_testDeposit.sol:e_testDeposit =======
Gas estimation:
construction:
   99 + 46200 = 46299
external:
   deposit(bytes32):	2014


### 단계 3: 컨트랙 배포

* 위 abi, bin을 복사해서 사용
* 아래는 eth를 사용해서 transactionHash, contractAddress 가져온다.
    * transactionHash:
"0xdd9182988b54e6e5587295a3cf3c82567ac99050804b6353918055d5d22b95fc"
    * contractAddress:
"0xf957f93a7b95c28005af20b5f8f396ba8b6f18d0"


In [11]:
%%writefile src/e_testDeposit.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_id","type":"bytes32"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":true,"name":"_id","type":"bytes32"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"Deposit","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);
var _bin="0x"+"6080604052348015600f57600080fd5b5060e78061001e6000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063b214faa514603e575b600080fd5b606760048036036020811015605257600080fd5b81019080803590602001909291905050506069565b005b803373ffffffffffffffffffffffffffffffffffffffff167f19dacbf83c5de6658e14cbf7bcae5c15eca2eedecf1c66fbca928e4d351bea0f346040518082815260200191505060405180910390a35056fea165627a7a72305820b8fc5d5b94e85b71dc3a6ce1672f4d4a7a24d46f3447825a14203c6f08f489a80029";
var _contract = web3.eth.contract(_abiArray);
var _instance=_contract.new({data:_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
  if (!err && contract.address)
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});
console.log("after async - transactionHash: ",_instance.transactionHash)
console.log("after async - contractAddress: ",_instance.address);

Overwriting src/e_testDeposit.js


In [12]:
!node src/e_testDeposit.js

after async - transactionHash:  null
after async - contractAddress:  undefined
transactionHash:  0x2c3250d1652bf0de46338aa6bcb17860bea29fa25a1c91f1a7ec64f261563183
contractAddress:  0x5b037cd00a6be142cf669d0863d44e91a7e90d98
transactionHash:  0x2c3250d1652bf0de46338aa6bcb17860bea29fa25a1c91f1a7ec64f261563183


### 단계 4: 사용

* event는 async방식으로 호출되면 실행된다.
* 호출한 클라이언트에 event의 argument를 받아볼 수 있다.
    * 예: result.args._from은 주소

currentProvider는 MetaMask, Mist와 같은 Wallet을 사용하는 경우 자동으로 설정된다.

In [13]:
%%writefile src/e_testDepositUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    // set the provider you want from Web3.providers
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
//web3.isConnected;
console.log(web3.eth.blockNumber);
var _abiArray=[{"constant":false,"inputs":[{"name":"_id","type":"bytes32"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":true,"name":"_id","type":"bytes32"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"Deposit","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);
var _address = '0x5b037cd00a6be142cf669d0863d44e91a7e90d98';
var _contract=web3.eth.contract(_abiArray);
var _instance=_contract.at(_address);
web3.eth.defaultAccount=web3.eth.accounts[0];
var event = _instance.Deposit(function (error, result) {
    if (!error) {
        console.log("event deposit ===> " + result.args._from + 
                    result.args._id + result.args._to);
        console.log("result: ", result);
        process.exit(1);
    }
});
_instance.deposit.sendTransaction({from:web3.eth.accounts[0]});


Overwriting src/e_testDepositUse.js


In [14]:
!node src/e_testDepositUse.js

99
event deposit ===> 0x21c704354d07f804bab01894e8b4eb4e0eba74510x7b2266726f6d223a223078323163373034333534643037663830346261623031undefined
result:  { address: '0x5b037cd00a6be142cf669d0863d44e91a7e90d98',
  blockNumber: 100,
  transactionHash: '0x73d4a85338b84b673b93b035d6609c0ce0e3175cbfb0978a09fef4bae1fcb724',
  transactionIndex: 0,
  blockHash: '0x40841fbf25ae63a16d3f09d2d8bda3b8fbcb695c278cc8b5081be528c09b4e39',
  logIndex: 0,
  removed: false,
  event: 'Deposit',
  args: 
   { _from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     _id: '0x7b2266726f6d223a223078323163373034333534643037663830346261623031',
     _value: { [String: '0'] s: 1, e: 0, c: [Object] } } }


## 실습 Multiply7

앞서 Multiply7을 구현해 보았다. 
이벤트를 추가해서 multiply()를 실행해 보자.
함수 내에서 변수 값을 확인하면서 디버깅 목적으로 **이벤트**가 사용되기도 한다.

### 단계 1: 컨트랙 개발

In [90]:
%%writefile src/Multiply7Event.sol
pragma solidity ^0.5.0;
contract Multiply7Event {
    event Print(uint);
    function multiply(uint input) public returns(uint) {
        uint res=input*7;
        emit Print(res);
        return res;
   }
}

Overwriting src/Multiply7Event.sol


### 단계 2: 컴파일

In [91]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   99 + 50800 = 50899
external:
   multiply(uint256):	1365
Binary: 
608060405234801561001057600080fd5b5060fe8061001f6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114604a575b600080fd5b607360048036036020811015605e57600080fd5b81019080803590602001909291905050506089565b6040518082815260200191505060405180910390f35b6000806007830290507f24abdb5865df5079dcc5ac590ff6f01d5c16edbc5fab4e195d9febd1114503da816040518082815260200191505060405180910390a18091505091905056fea165627a7a7230582046cff98a35c11661d6274a85a3b3cf0bb0a0cb33787b6fd675af2741a7009a130029
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint

### 단계 3: 컨트랙 배포
* 위 abi, bin을 복사해서 붙여 넣는다.

In [92]:
%%writefile src/Multiply7EventDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint256"}],"name":"Print","type":"event"}];
var _bin="608060405234801561001057600080fd5b5060fe8061001f6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114604a575b600080fd5b607360048036036020811015605e57600080fd5b81019080803590602001909291905050506089565b6040518082815260200191505060405180910390f35b6000806007830290507f24abdb5865df5079dcc5ac590ff6f01d5c16edbc5fab4e195d9febd1114503da816040518082815260200191505060405180910390a18091505091905056fea165627a7a7230582046cff98a35c11661d6274a85a3b3cf0bb0a0cb33787b6fd675af2741a7009a130029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Overwriting src/Multiply7EventDeploy.js


In [93]:
!node src/Multiply7EventDeploy.js

contractAddress:  undefined
transactionHash:  0xbab1ec9036f59e5f46a58365c320d2968a5dae30dffeae830146dc63c1cc137b
contractAddress:  0xee7b78482383b57db71c9c311be3a7ec682042a7
transactionHash:  0xbab1ec9036f59e5f46a58365c320d2968a5dae30dffeae830146dc63c1cc137b


### 단계 4: 사용

```m7.Print(function (error, result)``` 명령문으로 이벤트를 리스닝하고, 이벤트가 발생하는 경우, 그 결과 ```result```를 출력하고 있다.
이 이벤트는 ```multiply()``` 함수 내에서 발생되고 있다.
그런 까닭에 **```multiply()``` 함수는 ```pure```로 선언할 수 있지만 ```event```를 발생하기 때문에 그렇지 못했다**.
따라서 ```web3```에서 ```transaction```으로 호출해야 하지만. 아래 예제는 **```call()```로 호출**하였다.
```call()``` 함수는 트랜잭션을 발생시키지 않으므로 결과는 알 수 있지만, **이벤트는 발생하지 않는다**.

In [1]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint256"}],"name":"Print","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var m7 = _contract.at("0xee7b78482383b57db71c9c311be3a7ec682042a7");
var event = m7.Print(function (error, result) {
    if (!error) {
        console.log("event Print ===> " + result.args.res);
        console.log("result: ", result);
        process.exit(1);
    }
});
console.log(m7.multiply.call(8).toNumber());

Overwriting src/Multiply7EventUse.js


In [ ]:
!node src/Multiply7EventUse.js

56


In [2]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint256"}],"name":"Print","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var m7 = _contract.at("0xee7b78482383b57db71c9c311be3a7ec682042a7");
var event = m7.Print(function (error, result) {
    if (!error) {
        console.log("event Print ===> " + result.args.res);
        console.log("result: ", result);
        process.exit(1);
    }
});
console.log(m7.multiply.sendTransaction(8, {from:web3.eth.accounts[0]}));

Overwriting src/Multiply7EventUse.js


```sendTransaction()``` 함수로 트랜잭션을 발생시켜야 블록체인에 기록이되고, 그 로그에서 이벤트가 발생이 된다.

In [3]:
!node src/Multiply7EventUse.js

0xb6d5172a1c18f77b3976fa30f24496c805502edb94ed9f564a61da7510b6316b
event Print ===> undefined
result:  { address: '0xee7b78482383b57db71c9c311be3a7ec682042a7',
  blockNumber: 330,
  transactionHash: '0xb6d5172a1c18f77b3976fa30f24496c805502edb94ed9f564a61da7510b6316b',
  transactionIndex: 0,
  blockHash: '0xdd521f6c48f0f95eea5f4b132fbb0de31f78e0e9049c320020ba0c26a0bf2b62',
  logIndex: 0,
  removed: false,
  event: 'Print',
  args: { '': { [String: '56'] s: 1, e: 1, c: [Object] } } }


## 실습: 복수의 이벤트 사용하기
<a id='e-11'></a>

컨트랙에 이벤트를 여러 개 가질 수 있는지의 문제이다.
이벤트 **오버로딩**도 가능한지 해보자.

### 단계 1: 컨트랙 개발
* 출처 Ethereum wiki

```Test()```함수는 b를 변경하고
그 변경된 값을 ```Event1()```, ```Event2()```함수에서 사용하였다.
```Event1()```, ```Event2()```의 인자가 서로 다르게 설정되어 있다.
변수 ```b```는 ```bytes16```으로 선언되어있다. 값을 입력할 경우 16진수라서 32자리를 넣었다.

In [14]:
%%writefile src/e_testEvent.sol
pragma solidity ^0.5.0;
contract e_testEvent {
    bytes16 b;
    event Event();
    event Event1(string a, bytes16 b);
    event Event2(uint indexed a, bytes16 b);
    function foo() public { emit Event(); }
    function foo1(string memory a) public { emit Event1(a, b); }
    function foo2(uint a) public { emit Event2(a, b); }
    function Test() public { b = 0x12345678901234567890123456789012; }
}

Overwriting src/e_testEvent.sol


### 단계 2: 컴파일

In [15]:
!solc --abi src/e_testEvent.sol


======= src/e_testEvent.sol:e_testEvent =======
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"a","type":"string"}],"name":"foo1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"a","type":"uint256"}],"name":"foo2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"Test","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"foo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[],"name":"Event","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"a","type":"string"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event1","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"a","type":"uint256"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event2","type":"event"}]


In [16]:
!solc --bin src/e_testEvent.sol


======= src/e_testEvent.sol:e_testEvent =======
Binary: 
608060405234801561001057600080fd5b50610368806100206000396000f3fe608060405234801561001057600080fd5b5060043610610069576000357c0100000000000000000000000000000000000000000000000000000000900480630d5a88e31461006e5780637d5f3b3514610129578063a163a62414610157578063c298557814610161575b600080fd5b6101276004803603602081101561008457600080fd5b81019080803590602001906401000000008111156100a157600080fd5b8201836020820111156100b357600080fd5b803590602001918460018302840111640100000000831117156100d557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f82011690508083019250505050505050919291929050505061016b565b005b6101556004803603602081101561013f57600080fd5b8101908080359060200190929190505050610243565b005b61015f6102af565b005b61016961030e565b005b7ff3834da63f48008a317a12dfb256979c1a39f02d84ae95367e84625468f4b580816000809054906101000a9004700100000000000000000000000000000000026040518080602001836f

### 단계 3: 컨트랙 배포

* 위 abi, bin을 복사해서 넣는다.
* 계정을 unlock해야 한다.

```
> personal.unlockAccount(eth.accounts[0]);
Unlock account 0x21c704354d07f804bab01894e8b4eb4e0eba7451
Passphrase: 
```

In [1]:
%%writefile src/e_testEvent0.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"a","type":"string"}],"name":"foo1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"a","type":"uint256"}],"name":"foo2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"Test","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"foo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[],"name":"Event","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"a","type":"string"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event1","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"a","type":"uint256"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event2","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);
var _bin="0x"+"608060405234801561001057600080fd5b50610368806100206000396000f3fe608060405234801561001057600080fd5b5060043610610069576000357c0100000000000000000000000000000000000000000000000000000000900480630d5a88e31461006e5780637d5f3b3514610129578063a163a62414610157578063c298557814610161575b600080fd5b6101276004803603602081101561008457600080fd5b81019080803590602001906401000000008111156100a157600080fd5b8201836020820111156100b357600080fd5b803590602001918460018302840111640100000000831117156100d557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f82011690508083019250505050505050919291929050505061016b565b005b6101556004803603602081101561013f57600080fd5b8101908080359060200190929190505050610243565b005b61015f6102af565b005b61016961030e565b005b7ff3834da63f48008a317a12dfb256979c1a39f02d84ae95367e84625468f4b580816000809054906101000a9004700100000000000000000000000000000000026040518080602001836fffffffffffffffffffffffffffffffff19168152602001828103825284818151815260200191508051906020019080838360005b838110156102055780820151818401526020810190506101ea565b50505050905090810190601f1680156102325780820380516001836020036101000a031916815260200191505b50935050505060405180910390a150565b807f672d1aedf347b9d9982314a48e91caa3aad54cb8964e7694eb445a88f9723d0b6000809054906101000a90047001000000000000000000000000000000000260405180826fffffffffffffffffffffffffffffffff1916815260200191505060405180910390a250565b6f12345678901234567890123456789012700100000000000000000000000000000000026000806101000a8154816fffffffffffffffffffffffffffffffff021916908370010000000000000000000000000000000090040217905550565b7f57050ab73f6b9ebdd9f76b8d4997793f48cf956e965ee070551b9ca0bb71584e60405160405180910390a156fea165627a7a72305820f76034aad17b1c0f27de86bc70cf18a82bd17b67ef52935d4ac6371ef5c8abb70029";
var _contract = web3.eth.contract(_abiArray);
var _instance=_contract.new({data:_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
  if (!err && contract.address)
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});

Overwriting src/e_testEvent0.js


In [2]:
!node src/e_testEvent0.js

transactionHash:  0xa0c6668e78ce8d5c7636fd2e1d27124042d0f93db0258a4fe4e087ffb9acd2cd
contractAddress:  0x96d8b65b4586b090840d6587c216d4a69071f405
transactionHash:  0xa0c6668e78ce8d5c7636fd2e1d27124042d0f93db0258a4fe4e087ffb9acd2cd


* 위 ```transactionHash```를 가지고 ```contractAddress```를 찾는다.
    _address="0x96d8b65b4586b090840d6587c216d4a69071f405";

In [1]:
!geth --exec "eth.getTransactionReceipt('0xa0c6668e78ce8d5c7636fd2e1d27124042d0f93db0258a4fe4e087ffb9acd2cd')" attach http://117.16.44.45:8445

{
  blockHash: "0xfce7f2008aa67527c2058fe98e6ea6732b346859f369c79924fdd995ea02c217",
  blockNumber: 91,
  contractAddress: "0x96d8b65b4586b090840d6587c216d4a69071f405",
  cumulativeGasUsed: 280452,
  from: "0x21c704354d07f804bab01894e8b4eb4e0eba7451",
  gasUsed: 280452,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0x9f04f01e8b9c37675ad32f479d063e7d7988b121ac6fcf5335ecd57dd470aa50",
  to: null,
  transactionHash: "0xa0c6668e78ce8d5c7636fd2e1d27124042d0f93db0258a4fe4e087ffb9acd2cd",
  trans

### 단계 4: 사용

In [2]:
%%writefile src/e_testEvent0Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"a","type":"string"}],"name":"foo1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"a","type":"uint256"}],"name":"foo2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"Test","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"foo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[],"name":"Event","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"a","type":"string"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event1","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"a","type":"uint256"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event2","type":"event"}];
var e_testContract = web3.eth.contract(_abiArray);
//                              0x96d8b65b4586b090840d6587c216d4a69071f405
var e_test = e_testContract.at("0x96d8b65b4586b090840d6587c216d4a69071f405");
var event = e_test.Event(function (error, result) {
    if (!error) {
        console.log("e_test: event triggered =====> ",result);
        process.exit(1);
    }
});
console.log(e_test.foo({from:web3.eth.accounts[0],gas:100000}));

Writing src/e_testEvent0Use.js


In [4]:
!node src/e_testEvent0Use.js

0x4cf293174217fb0235d27ae32bea25b2bb14e94a0a4a2ffa0842270d02a43e97
e_test: event triggered =====>  { address: '0x96d8b65b4586b090840d6587c216d4a69071f405',
  blockNumber: 92,
  transactionHash: '0x4cf293174217fb0235d27ae32bea25b2bb14e94a0a4a2ffa0842270d02a43e97',
  transactionIndex: 0,
  blockHash: '0x58cb83818b9e079b487d50f814c4ffd550c8a990e595c681528407d299d1945f',
  logIndex: 0,
  removed: false,
  event: 'Event',
  args: {} }


event가 발생하려면, 로그에 기록이 되어야 하므로 ```sendTransaction()``` 함수를 사용한다 

아래 callback 함수 ```Event```, ```Event1``` 2개를 하나의 프로그램에서 호출하면 1개만 실행되어 주의가 필요하다.
```Event1```은 인자가 있지만 callback 함수는 변경이 없다. 즉 인자를 넣어서 이벤트 callback을 하지 않는다.


In [10]:
%%writefile src/e_testEvent0Use1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"a","type":"string"}],"name":"foo1","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"a","type":"uint256"}],"name":"foo2","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"Test","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"foo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[],"name":"Event","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"a","type":"string"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event1","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"a","type":"uint256"},{"indexed":false,"name":"b","type":"bytes32"}],"name":"Event2","type":"event"}];
var e_testContract = web3.eth.contract(_abiArray);
//                              0x96d8b65b4586b090840d6587c216d4a69071f405
var e_test = e_testContract.at("0x96d8b65b4586b090840d6587c216d4a69071f405");
/*var event = e_test.Event(function (error, result) {
    if (!error) {
        console.log("e_test: event triggered =====> ",result);
        process.exit(1);
    }
});
console.log(e_test.foo.sendTransaction({from:web3.eth.accounts[0],gas:100000}));*/
var event = e_test.Event1(function (error, result) {
    if (!error) {
        console.log("e_test: event1 triggered =====> ",result);
        process.exit(1);
    }
});
console.log(e_test.Test.sendTransaction({from:web3.eth.accounts[0],gas:100000}));
console.log(e_test.foo1.sendTransaction("hello from foo1",{from:web3.eth.accounts[0],gas:100000}));


Overwriting src/e_testEvent0Use1.js


In [11]:
!node src/e_testEvent0Use1.js

0x349832b21cdf5a95d1ef18e6028a6df015464588d206f3f63ee9971f73e09051
0x745c2618f76bd60d5ddc6e7d3bb43f476f5116defef9b409f65bec0ad2896f5b
e_test: event1 triggered =====>  { address: '0x96d8b65b4586b090840d6587c216d4a69071f405',
  blockNumber: 97,
  transactionHash: '0xa10b90425d8c78a8aa7feca94af1ee12517b2c589ba9e01840eb0cce268b864e',
  transactionIndex: 2,
  blockHash: '0xe258501df975bdb75a583ebef130f879e3717503f0d06e59f6f2861d942542da',
  logIndex: 1,
  removed: false,
  event: 'Event1',
  args: 
   { a: 'hello from foo1',
     b: '0x1234567890123456789012345678901200000000000000000000000000000000' } }


## 10.5 fallback
<a id='S.3.4'></a>

fallback() 함수는 보통 함수와는 다른 기능을 가진다.
사용자가 호출하지 않는다. 따라서 **이름, 인자와 반환 값이 없는 함수**이다.
또한 컨트랙에 **하나의 fallback 함수**만 존재할 수 있다.

fallback함수가 없으면 **예외**가 발생하게 된다. 
fallback이 호출되는 경우는:
* 사용자가 호출한 함수가 **존재하지 않을 경우**에 호출된다.
아래에서 callA() 함수를 호출해야 하는데 잘 못하여 callB()를 호출했다고 하자.
그러면 존재하지 않는 callB()를 실행하지 못하고 fallback()을 실행한다.
* 또한 ether를 **송금했는데 데이터를 비워서 호출**하게 되면 fallback()이 호출된다.



## 실습: fallback

존재하지 않는 함수를 호출해서 강제적으로 fallback 함수를 실행해 본다.

### 단계 1: 컨트랙 개발

In [1]:
%%writefile src/FallbackTest.sol
pragma solidity ^0.5.0;
contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    function() external {
        emit PrintLog("fallback called");
    }
}

Writing src/FallbackTest.sol


### 단계 2: 컴파일

In [3]:
!solc --gas --abi --bin -- src/FallbackTest.sol


======= src/FallbackTest.sol:FallbackTest =======
Gas estimation:
construction:
   129 + 82800 = 82929
external:
   fallback:	1729
   callA():	infinite
Binary: 
608060405234801561001057600080fd5b5061019e806100206000396000f3fe608060405234801561001057600080fd5b5060043610610048576000357c010000000000000000000000000000000000000000000000000000000090048063e7f09e05146100b2575b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100ba610135565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100fa5780820151818401526020810190506100df565b50505050905090810190601f1680156101275780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040805190810160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea165627a7a72305820eebb5fe6a91995

### 단계 3: 컨트랙 배포

In [4]:
%%writefile src/FallbackTestDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"callA","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"}];
var _bin="608060405234801561001057600080fd5b5061019e806100206000396000f3fe608060405234801561001057600080fd5b5060043610610048576000357c010000000000000000000000000000000000000000000000000000000090048063e7f09e05146100b2575b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100ba610135565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100fa5780820151818401526020810190506100df565b50505050905090810190601f1680156101275780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040805190810160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea165627a7a72305820eebb5fe6a9199558a5ae72300540824638cef5eb55bc15bba28146b1b2477f830029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0]}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/FallbackTestDeploy.js


In [5]:
!node src/FallbackTestDeploy.js

contractAddress:  undefined
transactionHash:  0xdb494cdded3d974b673855f6443744b5a03a893c9f94e5b216e8e57d895a3126
contractAddress:  0x3ab1841c257571342e04b0fc898d2e63050585b2
transactionHash:  0xdb494cdded3d974b673855f6443744b5a03a893c9f94e5b216e8e57d895a3126


### 단계 4: 배포

In [3]:
%%writefile src/FallbackTestUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"callA","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var fallback = _contract.at("0x3ab1841c257571342e04b0fc898d2e63050585b2");
var filter = fallback.PrintLog(function (error, result) {
  if (!error)
    console.log(result);
});
web3.eth.sendTransaction({from:web3.eth.accounts[0], to:"0x3ab1841c257571342e04b0fc898d2e63050585b2"});

Overwriting src/FallbackTestUse.js


#### 존재하지 않는 function selector

**존재하지 않는 함수를 호출**하면 fallback함수가 호출된다.
클라이언트에서 web3를 사용하면, 존재하지 않는 함수를 호출할 수 있는 방법은 없다.
가능한 방법은 ```web3.eth.sendTransaction()```의 ```data```, 즉 ```msg.data```를 수정하면 된다. 앞서 설명한 **function selector** (호출할 함수시그니처의 sha3 해시값으로 만들고, 앞 4바이트)가 존재하지 않도록 수정해야 한다.
```python
<address>.call(bytes4(bytes32(sha3("baz(uint32,bool)")))
```

아쉬운대로 다음과 같이 **함수명을 생략하여 강제적으로 존재하지 않는 함수**를 호출한다.
즉 컨트랙 주소로 그냥 트랜잭션을 전송하여 없는 함수가 호출된 것처럼 한다.

```python
web3.eth.sendTransaction({from:web3.eth.accounts[0], to:<<contract address>>});
```

In [ ]:
!node src/FallbackTestUse.js

{ address: '0x3ab1841c257571342e04b0fc898d2e63050585b2',
  blockNumber: 45484,
  transactionHash: '0x5725425ea888cfcad16298c2e4a1d3fd71cbb2880230b82762ac5be12fee7def',
  transactionIndex: 0,
  blockHash: '0xe4e9f371bb90f67279e9284c17451cba53b5eb850ed416249fe32b64f801e20c',
  logIndex: 0,
  removed: false,
  event: 'PrintLog',
  args: { '': 'fallback called' } }


## 실습: payable fallback

앞서 존재하지 않는 함수를 호출하는 경우, ```fallback```함수가 호출되었다.
또한 금액없이 송금을 하여도 ```fallback```함수가 호출된다.

송금을 하면서 오류가 발생하여 ```fallback```이 호출된다 하더라도 송금액은 받을 수 없다.
단, ```payable fabllback```으로 선언하면 송금액을 받을 수 있다.

```python
function () payable {}
```


### 단계 1: 컨트랙 개발

In [37]:
%%writefile src/MathMultiply7.sol
pragma solidity 0.4.21;

contract Multiply7 {
   event Print(uint);
   function () payable {}
   function multiply(uint input) public view returns (uint) {
      Print(input * 7);
      return input * 7;
   }
}

contract Math {
    Multiply7 m7=new Multiply7();
    function deposit(uint amount) payable public {
        require(msg.value==amount);
    }
    function m7set(address _addr) { m7 = Multiply7(_addr); }
    function multiply() public view returns(uint) {
        uint res=m7.multiply(8);
        //this.send11(); It does not send value.
        return res;
    }
    function send11() public returns(uint) {
        //m7.multiply.value(11)(9);
        m7.call.value(11)();
    }
    function queryBalance() public view returns(uint) {
        return address(this).balance;
    }
    function queryBalanceM7() public view returns(uint) {
        return address(m7).balance;
    }
}

Writing src/MathMultiply7.sol


### 단계 2: 컴파일

설치된 컴파일러 버전과 다른 0.4.21을 사용하기 위해 REMIX를 사용한다.

### 단계 3: 컨트랙 배포

In [45]:
%%writefile src/MathMultiply7Deploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
//taken from REMIX from here
var mathContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"send11","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"queryBalanceM7","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_addr","type":"address"}],"name":"m7set","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]);
var math = mathContract.new(
   {
     from: web3.eth.accounts[0], 
     data: '0x606060405261000c610071565b604051809103906000f080151561002257600080fd5b6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550341561006c57600080fd5b610081565b604051610105806103e883390190565b610358806100906000396000f300606060405260043610610078576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806336f40c611461007d5780633c58d09e146100a6578063b6b55f25146100cf578063b76d4d06146100e7578063f3593cd014610110578063fde3137f14610139575b600080fd5b341561008857600080fd5b610090610172565b6040518082815260200191505060405180910390f35b34156100b157600080fd5b6100b9610191565b6040518082815260200191505060405180910390f35b6100e560048080359060200190919050506101e5565b005b34156100f257600080fd5b6100fa6101f6565b6040518082815260200191505060405180910390f35b341561011b57600080fd5b610123610236565b6040518082815260200191505060405180910390f35b341561014457600080fd5b610170600480803573ffffffffffffffffffffffffffffffffffffffff169060200190919050506102e9565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16600b60405160006040518083038185875af1925050505090565b80341415156101f357600080fd5b50565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631905090565b6000806000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1663c6888fa160086040518263ffffffff167c010000000000000000000000000000000000000000000000000000000002815260040180828152602001915050602060405180830381600087803b15156102c957600080fd5b5af115156102d657600080fd5b5050506040518051905090508091505090565b806000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550505600a165627a7a72305820ca83e1f4743e5ee693d006254f4b0cb550b1e449bc6a95c89e2b575f901cd26300296060604052341561000f57600080fd5b60e88061001d6000396000f300606060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146041575b005b3415604b57600080fd5b605f60048080359060200190919050506075565b6040518082815260200191505060405180910390f35b60007f24abdb5865df5079dcc5ac590ff6f01d5c16edbc5fab4e195d9febd1114503da600783026040518082815260200191505060405180910390a16007820290509190505600a165627a7a72305820f19d1151fc4778023048c2f305ffb042231305525da155ac10d2fdd0112e607b0029', 
     gas: '4700000'
   }, function (e, contract){
    //console.log(e, contract);
    if (typeof contract.address !== 'undefined') {
         console.log('Contract mined! address: ' + contract.address + ' transactionHash: ' + contract.transactionHash);
    }
 })

Overwriting src/MathMultiply7Deploy.js


In [46]:
!node src/MathMultiply7Deploy.js

Contract mined! address: 0xf803dac95ea40f03736060114dfb9cca9ee7d514 transactionHash: 0x9551f9ea5a47180048c2ffd069586a934bb1c29e616712b944b228228c941966


### 단계 4: 사용

9를 곱하면서 ```11 wei```를 송금하려면 ```m7.multiply.value(11)(9)```로 호출할 수 있다.
송금하면서 ```payable fabllback```를 호출되게 하기 위해 의도적으로 아래와 같이 존재하지 않는 함수를 호출하였다.

```python
call.value()();
```

송금하려면 물론 ```deposit()```을 먼저해야 한다.

1. ```deposit(123)```
value를 반드시 함수인자와 동일하게 입력해야 한다.
REMIX에서 할 경우에도 마찬가지이다. ```Run``` 탭 상단의 ```value```, ```Deployed Contracts``` 함수의 인자 두 필드에 동일한 금액을 넣어준다.
2. ```queryBalance()``` --> 123 입금하고 난 후의 잔고
3. ```queryBalanceM7()``` ---> 0
4. ```send11()```
존재하지 않는 함수를 호출하여 ```m7.call.value(11)()```,
다른 컨트랙 ```Multiply7```의 ```payable fallback``` 함수를 호출하여 ```11 Wei```를 송금한다.
5. ```queryBalance()``` ---> 112
6. ```queryBalanceM7()``` ---> 11 ```payable fallback``` **존재하지 않는 함수를 호출하는 오류에도 불구하고 11 wei 송금 성공**

```python
> var math = mathContract.at("0xf803dac95ea40f03736060114dfb9cca9ee7d514");
undefined
> math.deposit.sendTransaction(123,{from:web3.eth.accounts[0], value:123});   value와 인자를 동일하게 123으로 입금
'0x9aa405b5780c999755ae85c3bc852a1e3b09a7d29b6fea20cdfb1af2ac4da19d'          REMIX에서도 마찬가지로 value와 인자를 일치시킴.
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> math.queryBalance.call().toNumber();                                        마이닝하고 나면 잔고 123
123
> math.queryBalanceM7.call().toNumber();                                      Multiply7 잔고는 0
0
> math.send11({from:web3.eth.accounts[0]});                                   call.value(11)로 송금
'0x9c497939df2a5d37cc7f9bf29b0f149831b82bfb841c2163badd360cba5f3360'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> math.queryBalance.call().toNumber();                                        마이닝하고 나면 잔고 112=123-11
112
> math.queryBalanceM7.call().toNumber();                                      Multiply7 잔고는 11증가
11
> math.multiply.call().toNumber();                                            함수내 8은 이미 입력되어 있으므로 56 = 8 x 7
56
```

In [4]:
%%writefile src/MathMultiply7Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var mathContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"send11","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"queryBalanceM7","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_addr","type":"address"}],"name":"m7set","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]);
var math = mathContract.at("0xf803dac95ea40f03736060114dfb9cca9ee7d514");
console.log("deposit 123..."+math.deposit.sendTransaction(123,{from:web3.eth.accounts[0], value:123}));
console.log("math balance: "+math.queryBalance.call().toNumber());
math.send11({from:web3.eth.accounts[0]});
console.log("math balance: "+math.queryBalance.call().toNumber());
console.log("m7 balance: "+math.queryBalanceM7.call().toNumber());
math.multiply.call().toNumber();

Overwriting src/MathMultiply7Use.js


In [5]:
!node src/MathMultiply7Use.js

deposit 123...0x3302ab96350ac6847fd7720470d3fb8f33b1635e1a7360d6c3fa14435acfc79d
math balance: 224
math balance: 224
m7 balance: 22


비동기 방식이라 마이닝을 하지 않으면 잔고조회가 올바르지 않을 수 있다.
node로 한 줄씩 실행하고 마이닝을 해가면서 결과를 확인해야 한다.

In [6]:
!node src/MathMultiply7Use.js

deposit 123...0x9fb715bff96730b3ec4e5d0be39bc4b299495e36bdb2b05a318e55495f1fc6b7
math balance: 336
math balance: 336
m7 balance: 33
